# 資料預處理

In [76]:
import sys
import os
 
root_dir = os.path.abspath(os.path.join(os.path.abspath(''), '..'))
sys.path.insert(0, root_dir)

from probe_mark.dataset_Della.get_img import create_folders,get_img,make_folder,get_img_label,get_img_mask
# import sys
# import os
 
# sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))

In [78]:
import pandas as pd
# from get_img import create_folders,get_img,make_folder,get_img_label,get_image

import os

from datetime import datetime
nowdir = os.getcwd()# 現在目錄
nowpath=root_dir+"\\data\\processed"
current_date = datetime.now()# 抓取現在的日期
formatted_date = current_date.strftime("%Y%m%d")# 格式化日期為所需格式

folder_path=make_folder(path=nowpath, folder_name='rowdata',date=formatted_date)


資料夾已存在 d:\github\Probe-Mark\data\processed\rowdata_20240813


In [79]:

folder_name = 'Artificial_VR22_D4070_MW120F-HS'
base_output_dir=make_folder(path=folder_path, folder_name=folder_name)
# 讀取 Excel 檔案中的特定 sheet
file_path = root_dir+'\\data\\raw\\'+folder_name+'\\picture.xlsx'
sheet_name = '主要'
# 讀取指定 sheet，並跳過第一行，同時指定新的欄位名稱
df = pd.read_excel(file_path, sheet_name=sheet_name, header=1, names=['name', 'img'])
img_name_list=df['name']  # 資料夾名稱列表
# filtered_list = [name for name in img_name_list if 'blank' not in name]
# 建立資料夾儲存圖片
created_folders = create_folders(base_output_dir, img_name_list)
get_img(file_path, sheet_name, created_folders,'original')

已建立資料夾 d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS
資料夾 25-OD25-1 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD25-1
資料夾 25-OD25-2 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD25-2
資料夾 25-OD25-3 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD25-3
資料夾 25-OD50-1 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD50-1
資料夾 25-OD50-2 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD50-2
資料夾 25-OD50-3 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD50-3
資料夾 25-OD75-1 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD75-1
資料夾 25-OD75-2 已建立於 d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD7

In [80]:

# 讀取 Excel 檔案中的特定 sheet
sheet_name = '體積面積量測'
# # 建立資料夾儲存圖片
get_img(file_path, sheet_name, created_folders,'red_mark')


圖片提取和儲存完成。


In [81]:
# 使用範例
for i in range(len(created_folders)):
    image_path1 = created_folders[i] + '\\original.png'
    image_path2 = created_folders[i] + '\\red_mark.png'
    image_path3 = created_folders[i] + '\\mask.png'
    image_path4 = created_folders[i] + '\\label.png'
    get_img_mask(image_path1, image_path2, image_path3)
    get_img_label(image_path1, image_path2, image_path4)

Image saved to d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD25-1\mask.png
Image saved to d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD25-1\label.png
Image saved to d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD25-2\mask.png
Image saved to d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD25-2\label.png
Image saved to d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD25-3\mask.png
Image saved to d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD25-3\label.png
Image saved to d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD50-1\mask.png
Image saved to d:\github\Probe-Mark\data\processed\rowdata_20240813\Artificial_VR22_D4070_MW120F-HS\25-OD50-1\label.png
Image saved to d:\github\Probe-Mark\data\pro

# 資料訓練

In [10]:
# folder_path=r'D:\github\Probe-Mark\data\processed\rowdata_20240813'

In [82]:
import os

def find_png(folder_path,img_name):
    label_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file == img_name:
                label_paths.append(os.path.join(root, file))
    return label_paths


label_paths = find_png(folder_path,"label.png")
orig_paths = find_png(folder_path,"original.png")

len(label_paths),len(orig_paths)

(234, 234)

### 準備數據集

In [3]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [98]:

class SegmentationDataset(Dataset):
    def __init__(self, orig_paths, label_paths, transform=None):
        self.orig_paths = orig_paths
        self.label_paths = label_paths
        self.transform = transform

    def __len__(self):
        return len(self.orig_paths)

    def __getitem__(self, idx):
        orig_image = Image.open(self.orig_paths[idx]).convert("RGB")
        label_image = Image.open(self.label_paths[idx]).convert("L")

        if self.transform:
            orig_image = self.transform(orig_image)
            label_image = self.transform(label_image)
        # orig_image = (3,128,128)
        # label_image = (1,128,128)
        return orig_image, label_image

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

dataset = SegmentationDataset(orig_paths, label_paths, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


### 定義模型架構(以下都還沒寫好)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        # 定義UNet的各個層
        self.enc1 = self.conv_block(3, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        self.center = self.conv_block(512, 1024)
        self.dec4 = self.conv_block(1024, 512)
        self.dec3 = self.conv_block(512, 256)
        self.dec2 = self.conv_block(256, 128)
        self.dec1 = self.conv_block(128, 64)
        self.final = nn.Conv2d(64, 1, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # 前向傳播定義
        enc1 = self.enc1(x)
        enc2 = self.enc2(F.max_pool2d(enc1, 2))
        enc3 = self.enc3(F.max_pool2d(enc2, 2))
        enc4 = self.enc4(F.max_pool2d(enc3, 2))
        center = self.center(F.max_pool2d(enc4, 2))
        dec4 = self.dec4(F.interpolate(center, scale_factor=2, mode='bilinear', align_corners=True) + enc4)
        dec3 = self.dec3(F.interpolate(dec4, scale_factor=2, mode='bilinear', align_corners=True) + enc3)
        dec2 = self.dec2(F.interpolate(dec3, scale_factor=2, mode='bilinear', align_corners=True) + enc2)
        dec1 = self.dec1(F.interpolate(dec2, scale_factor=2, mode='bilinear', align_corners=True) + enc1)
        final = self.final(dec1)
        return torch.sigmoid(final)

model = UNet()


### 訓練模型

In [101]:
# epoch
# model = model.cuda()
for epoch in range(1):
    for orig_image, label_image in dataloader:
        orig_image = orig_image.cuda()
        label_image = label_image.cuda()

        # orig_image = orig_image.cpu()
        # label_image = label_image.cpu()
        print(orig_image.device, label_image.device)
        print(orig_image.shape, label_image.shape)
        # optmiizer.zero_grad()
        # output = model(orig_image)
        # loss = criterion(output, label_image)
        # loss.backward()
        # optimizer.step()
# orig_image

cuda:0 cuda:0
torch.Size([32, 3, 128, 128]) torch.Size([32, 1, 128, 128])
cuda:0 cuda:0
torch.Size([32, 3, 128, 128]) torch.Size([32, 1, 128, 128])
cuda:0 cuda:0
torch.Size([32, 3, 128, 128]) torch.Size([32, 1, 128, 128])
cuda:0 cuda:0
torch.Size([32, 3, 128, 128]) torch.Size([32, 1, 128, 128])
cuda:0 cuda:0
torch.Size([32, 3, 128, 128]) torch.Size([32, 1, 128, 128])
cuda:0 cuda:0
torch.Size([32, 3, 128, 128]) torch.Size([32, 1, 128, 128])
cuda:0 cuda:0
torch.Size([32, 3, 128, 128]) torch.Size([32, 1, 128, 128])
cuda:0 cuda:0
torch.Size([10, 3, 128, 128]) torch.Size([10, 1, 128, 128])


In [94]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')


cuda:0 cuda:0
torch.Size([234, 3, 1024, 1024]) torch.Size([234, 1, 1024, 1024])


### 模型推論
1. 打印loss
2. 語意分割結果

In [ ]:
model.eval()
with torch.no_grad():
    for images, labels in dataloader:
        outputs = model(images)
        # 在這裡可以處理或保存輸出結果
        # 例如: 保存結果圖片，計算評估指標等
